# SQL
SQL can be a powerful asset as a Data Scientist. In this week's Notebook, we will explore how we can use SQLAlchemy and Pandas to efficiently use SQL databases in our projects.

In [3]:
!conda install sqlalchemy

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /home/yabo/anaconda3:

The following NEW packages will be INSTALLED:

    libgcc-ng:  7.2.0-h7cc24e2_2     

The following packages will be UPDATED:

    conda:      4.3.29-py36ha26b0c0_0 --> 4.3.30-py36h5d9f9f4_0
    sqlalchemy: 1.1.9-py36_0          --> 1.1.13-py36hfb5efd7_0

Proceed ([y]/n)? ^C


In [2]:
import sqlalchemy
from sqlalchemy import (Table, Column, Integer, String, Float, Date,
                        MetaData, create_engine)
from sqlalchemy.sql import select
from datetime import date

In [3]:
# Setup a connection to our toy SQLite database
engine = create_engine('sqlite:///test.db')

   ### Example 1: SQLAlchemy as a SQL Engine
   This is just to show the "core" SQLAlchemy. I don't suggest you necessarily do things this way...

In [4]:
metadata = MetaData()

# Define SQL Schema
students = sqlalchemy.Table('student', metadata,
    Column('netid', String(8), primary_key=True),
    Column('first_name', String(100)),
    Column('last_name', String(100)))

# Create Table in our database
query = students.create(engine, checkfirst=True)

In [12]:
# Construct an insertion query
insert_query = students.insert().values(netid="bcongdo2",
                                        first_name="Ben",
                                        last_name="Congdon")

# The Query we will execute
#print 
insert_query.compile()
# The parameters to this query
#print 
insert_query.compile().params

{'first_name': 'Ben', 'last_name': 'Congdon', 'netid': 'bcongdo2'}

In [13]:
# Actually run the query
connection = engine.connect()
connection.execute(insert_query)

# Run some more insertions
connection.execute(students.insert(), [
        {'netid': 'abc1', 'first_name': 'john', 'last_name': 'doe'},
        {'netid': 'def2', 'first_name': 'jane', 'last_name': 'doe'}
    ])

In [15]:
# Simple "SELECT"
s = select([students]) # Equivilant to "SELECT * FROM students"
result = connection.execute(s)
for i in result:
    print(i)

('bcongdo2', 'Ben', 'Congdon')
('abc1', 'john', 'doe')
('def2', 'jane', 'doe')


In [17]:
# "SELECT" with parameters
# Equivilant to "SELECT netid FROM students WHERE first_name = "Ben""
s = select([students.c.netid]).where(students.c.first_name == "Ben")
for i in connection.execute(s):
    print(i)

('bcongdo2',)


Interested in learning more? Check out [this](http://docs.sqlalchemy.org/en/latest/core/tutorial.html) SQLAlchemy Expression Language tutorial.

### Example 2: SQLAlchemy as a Object Relational Mapping tool
When using an ORM, our SQL becomes even more "object" like, and we get to think less about the actual queries that are running. We are getting closer to a "in-language" database layer - but really, this is just providing an extra layer of abstraction.

In [18]:
# Random SQLAlchemy magic
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Our Movie class
class Movie(Base):
    __tablename__ = "movies"
    id = Column(Integer, primary_key=True)
    title = Column(String)
    rating = Column(Float)
    duration_minutes = Column(Integer)
    release_date = Column(Date)
    
# Tell SQLAlchemy to create the table
Base.metadata.create_all(engine)

In [19]:
# Now we can create and modify Movie objects
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

inception = Movie(title="Inception",
                  rating=8.8,
                  duration_minutes=148,
                  release_date=date(2010, 7, 16))
interstellar = Movie(title="Interstellar",
                     rating=8.6,
                     duration_minutes=169,
                     release_date=date(2014, 11, 4))

# Add our ORM objects to our session
session.add(inception)
session.add(interstellar)

# Tell SQLAlchemy to commit all changes to the database
session.commit()

In [21]:
# Queries
# Get all movies
for movie in session.query(Movie):
    print(movie.title, movie.id, movie.rating)

Inception 1 8.8
Interstellar 2 8.6


In [23]:
# Get movies with rating above 8.4
print(session.query(Movie).filter(Movie.rating > 8.4)[0].title)

Inception


Interested in learning more? Look [here](http://docs.sqlalchemy.org/en/rel_1_1/orm/tutorial.html) for a good SQLAlchemy ORM tutorial.

### Example 3: Pandas for SQL

In [24]:
import pandas as pd

# Read in a CSV to a dataframe
df = pd.read_csv('airplane_crashes.csv', encoding='utf-8')

# Export a dataframe to a SQL table
df.to_sql('airplane_crashes', connection, if_exists='replace')

In [25]:
# Load a SQL table into a dataframe
df_read = pd.read_sql('airplane_crashes', connection)
df_read

,index,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,None,Demonstration,Wright Flyer III,None,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,None,Test flight,Dirigible,None,None,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,2,08/06/1913,None,"Victoria, British Columbia, Canada",Private,-,None,Curtiss seaplane,None,None,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,3,09/09/1913,18:30,Over the North Sea,Military - German Navy,None,None,Zeppelin L-1 (airship),None,None,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,None,None,Zeppelin L-2 (airship),None,None,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...
5,5,03/05/1915,01:00,"Tienen, Belgium",Military - German Navy,None,None,Zeppelin L-8 (airship),None,None,41.0,21.0,0.0,Crashed into trees while attempting to land af...
6,6,09/03/1915,15:20,"Off Cuxhaven, Germany",Military - German Navy,None,None,Zeppelin L-10 (airship),None,None,19.0,19.0,0.0,"Exploded and burned near Neuwerk Island, when..."
7,7,07/28/1916,None,"Near Jambol, Bulgeria",Military - German Army,None,None,Schutte-Lanz S-L-10 (airship),None,None,20.0,20.0,0.0,"Crashed near the Black Sea, cause unknown."
8,8,09/24/1916,01:00,"Billericay, England",Military - German Navy,None,None,Zeppelin L-32 (airship),None,None,22.0,22.0,0.0,Shot down by British aircraft crashing in flames.
9,9,10/01/1916,23:45,"Potters Bar, England",Military - German Navy,None,None,Zeppelin L-31 (airship),None,None,19.0,19.0,0.0,Shot down in flames by the British 39th Home D...


In [26]:
# Execute SQL to get a dataframe
df_read = pd.read_sql('SELECT Date, Time, Type FROM airplane_crashes', connection)
df_read

,Date,Time,Type
0,09/17/1908,17:18,Wright Flyer III
1,07/12/1912,06:30,Dirigible
2,08/06/1913,None,Curtiss seaplane
3,09/09/1913,18:30,Zeppelin L-1 (airship)
4,10/17/1913,10:30,Zeppelin L-2 (airship)
5,03/05/1915,01:00,Zeppelin L-8 (airship)
6,09/03/1915,15:20,Zeppelin L-10 (airship)
7,07/28/1916,None,Schutte-Lanz S-L-10 (airship)
8,09/24/1916,01:00,Zeppelin L-32 (airship)
9,10/01/1916,23:45,Zeppelin L-31 (airship)
